In [4]:
from ANPR import ANPR
import gradio as gr
import imutils
import cv2

# Automatic License Plate Recognition

In this notebook I will build a computer vision based system for the automatic
recognition of license plate with the aim to build an automatic door system.

All the CV operation is developed and explained in the _ANPR_ class, and in this notebook
I will define how the interaction will be.

I will use gradio to show the operation of the license plate recognition system

In [5]:
def cleanup_text(text):
	# strip out non-ASCII text so we can draw the text on the image
	# using OpenCV
	return "".join([c if ord(c) < 128 else "" for c in text]).strip()


def plate_detection(img):
  anpr = ANPR(debug=False)
  img = imutils.resize(img, width=600)
  lp_text, lp_cnt = anpr.find_and_ocr(img)
  # only continue if the license plate was successfully OCR'd
  if lp_text is not None and lp_cnt is not None:
    # fit a rotated bounding box to the license plate contour and
    # draw the bounding box on the license plate
    box = cv2.boxPoints(cv2.minAreaRect(lp_cnt))
    box = box.astype("int")
    cv2.drawContours(img, [box], -1, (0, 255, 0), 2)
    # compute a normal (unrotated) bounding box for the license
    # plate and then draw the OCR'd license plate text on the
    # image
    (x, y, w, h) = cv2.boundingRect(lp_cnt)
    cv2.putText(img, cleanup_text(lp_text), (x, y - 15),
        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    # show the output ANPR image
    print("[INFO] {}".format(lp_text))
    cv2.imshow("Output ANPR", img)
    cv2.waitKey(0)

  return lp_text

output_text = gr.outputs.Textbox()
iface = gr.Interface(plate_detection, gr.inputs.Image(), 
                     outputs=output_text)
iface.launch()


Running locally at: http://127.0.0.1:7861/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7861/', None)